In [1]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # choose to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc

In [2]:
""" Function """
def ctrl_add(qc, a, b, ctrl_value, c):
    """
    Takes in a quantum circuit and two quantum registers (a and b),
    performs a controlled addition, and returns the updated circuit.

    #Requires n-1 ancillaries
    
    Parameters:
    - qc: QuantumCircuit on which the addition is performed
    - a: QuantumRegister for the first number
    - b: QuantumRegister for the second number
    - ctrl: Control qubit that determines if the ADD is performed
    - c: QuantumRegister for carry qubits
    
    Returns:
    - QuantumCircuit: The updated quantum circuit with the addition operation applied.
    """

    n = len(a)  # number of qubits per register


    ''' 
    INPUT CARRY/ OUTPUT carry
        This implementation is setup so the first seg is a repeated seg, so if we wanted 
        this adder function to handle input carry:
        1. add Cin as a parameter, 1. use CNOT to initialise c[0] (which otherwise remains 0)

        ### add c_in as a parameter and c_out ###
        #bit hard to do those because the uncomputes go back to the last called function for c_in and this call for c_out

        # Apply Gidney adder gates
    for i in range(1,n):
        qc.cx(c[i], a[i])
        qc.cx(c[i], b[i])
        qc.ccx(a[i], b[i], c[i])
        qc.cx(c[i], c[i])
        qc.ccx(ctrl[0], a[i], b[i])
        qc.cx(c[i], a[i])
        qc.cx(c[i], b[i])
    '''
        
   # First seg TempAND
    qc.ccx(a[0], b[0], c[0]) 

    # ctrled Gidney adder
    for i in range(1,n-1):
        qc.cx(c[i-1], a[i])
        qc.cx(c[i-1], b[i])
        qc.ccx(a[i], b[i], c[i])
        qc.cx(c[i-1], c[i])

    # Last Segment
    qc.cx(a[n-1],c[n-2])
    qc.ccx(ctrl_value, c[n-2], b[n-1])

    #un compute + last bits
    for i in range(n-2,0,-1): # i goes from n-2 to 1
        #uncompute
        qc.cx(c[i-1], c[i])
        qc.ccx(a[i], b[i], c[i])

        qc.ccx(ctrl_value, a[i], b[i])
        qc.cx(c[i-1], a[i])
        qc.cx(c[i-1], b[i])

    # End of first
    qc.ccx(a[0], b[0], c[0])
    qc.ccx(ctrl_value, a[0], b[0])

    
    return qc

def ctrl_add_cout(qc, a, b, ctrl_value, c, c_out): #This is for the last add
        """
        Takes in a quantum circuit and two quantum registers (a and b),
        performs a controlled addition WITH OUTPUT CARRY, and returns the updated circuit.
        
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - a: QuantumRegister for the first number
        - b: QuantumRegister for the second number
        - ctrl: Control qubit that determines if the ADD is performed
        - c: QuantumRegister for carry qubits
        - c_out: this is given as an individual qubit not a register, it holds the carry out
        
        Returns:
        - QuantumCircuit: The updated quantum circuit with the addition operation applied.
        """
    
        n = len(a)  # number of qubits per register
            
       # First seg TempAND
        qc.ccx(a[0], b[0], c[0]) 
    
        # ctrled Gidney adder
        for i in range(1,n-1):
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i-1], c[i])

        # Last Segment
        qc.cx(c[n-2], a[n-1])
        qc.cx(c[n-2], b[n-1])

        # need a 3- bit controlled gate for carry out
        qc.ccx(ctrl_value, a[n-1], c[n-1])
        qc.ccx(c[n-1], b[n-1], c_out) 
        qc.ccx(ctrl_value, a[n-1], c[n-1])
        
        qc.ccx(ctrl_value, c[n-2], c_out) # final gate on c_out
    
        qc.ccx(ctrl_value, a[n-1], b[n-1])
        qc.cx(c[n-2],a[n-1])
        qc.cx(c[n-2],b[n-1])
    
        #un compute + last bits
        for i in range(n-2,0,-1): # i goes from n-2 to 1
            #uncompute
            qc.cx(c[i-1], c[i])
            qc.ccx(a[i], b[i], c[i])
    
            qc.ccx(ctrl_value, a[i], b[i])
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
    
        # End of first
        qc.ccx(a[0], b[0], c[0])
        qc.ccx(ctrl_value, a[0], b[0])
        
        return qc

In [5]:
""" MAIN, This does ctrl a + b"""

# Number of bits
n = 4

ancillaries = QuantumRegister(n-1, 'ancillaries')
ctrl = QuantumRegister(1, 'ctrl')
a = QuantumRegister(n, 'a')
b = QuantumRegister(n, 'b')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(2 * n + 2*n, 'result')

qc = QuantumCircuit(a, b, ctrl, ancillaries, result)

"""initialize the quantum numbers"""
qc = init_random_num(qc, a)
qc = init_random_num(qc, b)
#qc.h(a[n-1])

""" # Initialize the ctrl # """
qc.x(ctrl[0])

#apply the ctrl add/sub operation
qc = ctrl_add(qc, a, b, ctrl[0], ancillaries)


# Measure the qubits into the classical register
qc.measure(a, result[:n])      # Measure 'a' into the first n classical bits
qc.measure(b, result[n:2*n])   # Measure 'b' into the last n classical bits
#qc.measure(ancillaries,[])

# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

print("Measurement results:")
for bitstring, count in counts.items():
    # bitstring is of the form 'result' where the first n bits are for 'a' and the last n bits are for 'b'
    a_result = bitstring[-n:]  # Last n bits for b
    b_result = bitstring[-2*n:-n]  # First n bits for a
    a_decimal_value = int(a_result, 2)
    b_decimal_value = int(b_result, 2)

    print(f"a = {a_result} (decimal: {a_decimal_value}), b = {b_result} (decimal: {b_decimal_value})")

print(counts)

a: 0010 (decimal: 2)
b: 0011 (decimal: 3)
{'0000000001010010': 1024}
Measurement results:
a = 0010 (decimal: 2), b = 0101 (decimal: 5)
{'0000000001010010': 1024}
